We replicate the ACTG-synthetic data generation process here: https://github.com/paidamoyo/counterfactual_survival_analysis/blob/main/actg_synthetic.ipynb. Most of the code were directly taken from their github repo.

In [1]:
import pandas as pd
import numpy as np

np.random.seed(2025511)

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

import sys
import os

### Load ACTG Data

In [2]:
#https://rdrr.io/cran/speff2trial/man/ACTG175.html
data_frame = pd.read_csv('ACTG175.csv', index_col=0)
print("head of data:{}, data shape:{}".format(data_frame.head(), data_frame.shape))
print("unique", len(np.unique(data_frame[['pidnum']])))

print(data_frame.columns)

#categorical = ['hemo, 'homo', 'drugs', 'oprior', 'z30', 'zprior', 'race', 'gender', 
#'str2', 'strat', 'symptom', 'treat','offtrt', 'r',  ]
# outcome = ['cens', 'days']
# treatment = 'arms'
# treatment arm (0=zidovudine, 1=zidovudine and didanosine, 2=zidovudine and zalcitabine, 3=didanosine)

to_drop = ['cens', 'days', 'arms', 'pidnum']



#print("head of x_data:", x_data.shape)
print("head of data:",  data_frame.shape)

head of data:   pidnum  age    wtkg  hemo  homo  drugs  karnof  oprior  z30  zprior  \
1   10056   48  89.813     0     0      0     100       0    0       1   
2   10059   61  49.442     0     0      0      90       0    1       1   
3   10089   45  88.452     0     1      1      90       0    1       1   
4   10093   47  85.277     0     1      0     100       0    1       1   
5   10124   43  66.679     0     1      0     100       0    1       1   

   preanti  race  gender  str2  strat  symptom  treat  offtrt  cd40  cd420  \
1        0     0       0     0      1        0      1       0   422    477   
2      895     0       0     1      3        0      1       0   162    218   
3      707     0       1     1      3        0      1       1   326    274   
4     1399     0       1     1      3        0      1       0   287    394   
5     1352     0       1     1      3        0      0       0   504    353   

   cd496  r  cd80  cd820  cens  days  arms  
1  660.0  1   566    324    

In [3]:
na_columns = ['cd496']
na_data = data_frame[na_columns]
print("na_data description:{}".format(na_data.describe()))

na_data description:          cd496
count  1342.000
mean    328.571
std     174.656
min       0.000
25%     209.250
50%     321.000
75%     440.000
max    1190.000


In [4]:
def print_missing_prop(covariates):
    missing = np.array(np.isnan(covariates), dtype=float)
    shape = np.shape(covariates)
    proportion = np.sum(missing) / (shape[0] * shape[1]) * 100
    print("missing_proportion:{}".format(proportion))
    

print_missing_prop(data_frame)

missing_proportion:1.3800148910013332


In [5]:
data_frame = data_frame.fillna(data_frame.median())

In [6]:
print(data_frame.isna().any())


pidnum     False
age        False
wtkg       False
hemo       False
homo       False
drugs      False
karnof     False
oprior     False
z30        False
zprior     False
preanti    False
race       False
gender     False
str2       False
strat      False
symptom    False
treat      False
offtrt     False
cd40       False
cd420      False
cd496      False
r          False
cd80       False
cd820      False
cens       False
days       False
arms       False
dtype: bool


In [7]:
age_data = data_frame[['age']]
print("age description:{}".format(age_data.describe()))
age_data =np.array(age_data).reshape(len(age_data))
print(age_data.shape)
mu_age = np.mean(age_data)

age description:            age
count  2139.000
mean     35.248
std       8.709
min      12.000
25%      29.000
50%      34.000
75%      40.000
max      70.000
(2139,)


In [8]:
cd40_data = data_frame[['cd40']]
print("cd40_data description:{}".format(cd40_data.describe()))
cd40_data=np.array(cd40_data).reshape(len(cd40_data))
print(cd40_data.shape)
mu_cd40 = np.mean(cd40_data) 

cd40_data description:           cd40
count  2139.000
mean    350.501
std     118.574
min       0.000
25%     263.500
50%     340.000
75%     423.000
max    1199.000
(2139,)


In [9]:
x_data =  data_frame.drop(labels=to_drop, axis=1)
print("covariate description:{}".format(x_data.describe()))
x_data =np.array(x_data).reshape(x_data.shape)
print(x_data.shape)

covariate description:            age      wtkg      hemo      homo     drugs    karnof    oprior  \
count  2139.000  2139.000  2139.000  2139.000  2139.000  2139.000  2139.000   
mean     35.248    75.125     0.084     0.661     0.131    95.446     0.022   
std       8.709    13.263     0.278     0.473     0.338     5.901     0.147   
min      12.000    31.000     0.000     0.000     0.000    70.000     0.000   
25%      29.000    66.679     0.000     0.000     0.000    90.000     0.000   
50%      34.000    74.390     0.000     1.000     0.000   100.000     0.000   
75%      40.000    82.555     0.000     1.000     0.000   100.000     0.000   
max      70.000   159.939     1.000     1.000     1.000   100.000     1.000   

            z30  zprior   preanti      race    gender      str2     strat  \
count  2139.000  2139.0  2139.000  2139.000  2139.000  2139.000  2139.000   
mean      0.550     1.0   379.176     0.288     0.828     0.586     1.980   
std       0.498     0.0   468.658  

### ACTG-Synthentic Data

We simulate potential outcomes according to a Gompertz-Cox distribution with selection bias from a simple logistic model for $P(A=1| X=x )$ and AFT-based censoring mechanism.  Below is our generative scheme: 

$$X = \text{ACTG covariates}$$
$$P(A=1|X=x) = \frac{1}{b} \times \left(a + \sigma\left( \eta ({\rm AGE} - \mu_{\rm AGE} + {\rm CD40} - \mu_{\rm CD40}) \right) \right)$$
$$ U  \sim {\rm Uniform} (0, 1 )$$
$$T_A  =  \frac{1}{\alpha_A} \log \left[1 - \frac{\alpha_A \log U}{ \lambda_A  \exp\left( x ^T  \beta_A\right)  }  \right]$$
$$\log C  \sim {\rm Normal} (\mu_c, \sigma_c^2)$$
$$Y = \min(T_A, C)$$

where $\{ \beta_A, \alpha_A, \lambda_A, b, a, \eta, \mu_c, \sigma_c \}$ are hyper-parameters and $ \{\mu_{\rm AGE},  \mu_{\rm CD40}\}$ are the means for age and CD40 respectively.

In [10]:
# Beta for T=1
#    age          wtkg          hemo          homo         drugs        karnof        oprior           z30        zprior       preanti 
#  0.0026987044  0.0094957416 -0.2047708817 -0.0518243280 -0.2168722467  0.0076266828 -0.0796099695  0.6258748940            NA  0.0009670592 
#          race        gender          str2         strat       symptom         treat        offtrt          cd40         cd420         cd496 
# -1.0101809693 -0.4038655688 -1.5959739338 -0.0563572096  0.5244218189            NA  0.2280296997  0.0035548596 -0.0047974742 -0.0121293815 
#             r          cd80         cd820 
# -1.0625208970 -0.0004266264  0.0005844290 

beta_one = [ 0.0026987044,  0.0094957416, -0.2047708817, -0.0518243280, -0.2168722467,  0.0076266828, -0.0796099695,  
            0.6258748940, 0, 0.0009670592, -1.0101809693, -0.4038655688, -1.5959739338, -0.0563572096, 0.5244218189,    
            0,  0.2280296997,  0.0035548596, -0.0047974742, -0.0121293815, -1.0625208970, -0.0004266264,0.0005844290 ]

beta_one = np.array(beta_one)
print("beta_one: ", beta_one.shape)

assert(beta_one.shape[0] == x_data.shape[1])






beta_one:  (23,)


In [11]:
## Beta for T=0

#          age          wtkg          hemo          homo         drugs        karnof        oprior           z30        zprior       preanti 
#  1.148569e-02  3.896347e-03 -3.337743e-02 -1.215442e-01 -6.036002e-01  4.563380e-03 -5.217492e-02  1.414948e+00            NA  9.294612e-06 
#          race        gender          str2         strat       symptom         treat        offtrt          cd40         cd420         cd496 
#  7.863787e-02  4.756738e-01 -7.807835e-01 -1.766999e-01  1.622865e-01            NA  1.551692e-01  2.793350e-03 -6.417969e-03 -9.856514e-03 
#             r          cd80         cd820 
# -1.127284e+00  2.247806e-04  1.952943e-04 


beta_zero = [1.148569e-02,  3.896347e-03, -3.337743e-02, -1.215442e-01, -6.036002e-01,  4.563380e-03, -5.217492e-02,
             1.414948e+00, 0,  9.294612e-06, 7.863787e-02,  4.756738e-01, -7.807835e-01, -1.766999e-01,  1.622865e-01,
             0,  1.551692e-01,  2.793350e-03, -6.417969e-03, -9.856514e-03,  -1.127284e+00, 
             2.247806e-04,  1.952943e-04] 
beta_zero = np.array(beta_zero)
print("beta_zero: ", beta_zero.shape)

assert(beta_zero.shape[0] == x_data.shape[1])


beta_zero:  (23,)


In [12]:
def sigmoid(a):
    return 1/(1 + np.exp(-a))

In [13]:
# random varibles for data (x, y, \delta, a)
N = x_data.shape[0]

T_F = np.zeros(N)
T_CF = np.zeros(N)
Y_F = np.zeros(N)
Y_CF = np.zeros(N)
delta_F = np.zeros(N)
delta_CF = np.zeros(N)
A =  np.zeros(N)
prop =  np.zeros(N)

T_1 = np.zeros(N)
T_0 = np.zeros(N)

time = 'days'
c_mean_time = 1000 # mean censoring time
c_std_time = 100 # std censoring time

lamd_zero = 6 * 1e-4
lamd_one = 6 * 1e-4
alpha = 0.0055

U_0 =  np.random.uniform(0,1, size=(N))
U_1 =  np.random.uniform(0,1, size=(N))
#C = np.random.uniform(c_start_time, c_end_time, size=N) # Non-Informative censoring
C = np.random.normal(c_mean_time, c_std_time, size=(N))
gamma = -30
b_zero = 0


for i in range(N):
    
    pos_age_i = age_data[i]
    beta_i = gamma * ((pos_age_i - mu_age) + (cd40_data[i]-mu_cd40))# counfounding
    #beta_i =  gamma * (pos_age_i - mu_age) 
    
    balance = 1.5 # parameter to balance
    prop_i = 1/balance * (0.3 + sigmoid(beta_i))
    prop[i] = prop_i
    
    A_i = np.random.binomial(n=1, p=prop_i, size=1)[0]
    A[i] = A_i
    
    cov_eff_T_0 = lamd_zero * np.exp(np.dot(x_data[i], beta_zero))
    cov_eff_T_1 = lamd_one * np.exp(np.dot(x_data[i], beta_one))
    
                       
    stoch_0 = alpha * np.log(U_0[i])
    stoch_1 = alpha * np.log(U_1[i])
    

    T_1_i = 1/alpha * np.log(1 - stoch_1/cov_eff_T_1) + b_zero
    T_0_i = 1/alpha * np.log(1 - stoch_0/cov_eff_T_0)  
    T_1[i] = T_1_i
    T_0[i] = T_0_i
    
    T_F_i =  A_i * T_1_i + (1-A_i) * T_0_i
    T_CF_i = (1-A_i) * T_1_i + A_i * T_0_i
    
    
    C_i = C[i]

    Y_F_i = min(T_F_i, C_i)
    Y_CF_i = min(T_CF_i, C_i)
    
    delta_F_i = T_F_i <= C_i
    delta_F[i] = delta_F_i
    
    delta_CF_i = T_CF_i <= C_i
    delta_CF[i] = delta_CF_i 
    
    T_F[i] = T_F_i
    T_CF[i] = T_CF_i
    
    
    Y_F[i] = Y_F_i
    Y_CF[i] = Y_CF_i
    



/tmp/ipykernel_1064482/2523217602.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-a))


### Organize the columns to match our experiments

In [14]:
# idx,observed_time,event,T0,T1,T,C,W,true_cate
actg_syn = pd.DataFrame({'idx': list(range(N)),
              'observed_time': Y_F / 30., # convert to months
              'event': delta_F,
              'T0': T_0 / 30.,
              'T1': T_1 / 30.,
              'T': T_F / 30.,
              'C': C / 30.,
              'W': A,
              'true_cate': (T_1 - T_0) / 30.
              })
actg_syn = pd.concat([actg_syn, data_frame.drop(labels=to_drop, axis=1).reset_index(drop=True)], axis = 1)
actg_syn.to_csv('actg_syn.csv', index=False)
actg_syn

,idx,observed_time,event,T0,T1,T,C,W,true_cate,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,zprior,preanti,race,gender,str2,strat,symptom,treat,offtrt,cd40,cd420,cd496,r,cd80,cd820
0,0,35.116,0.0,53.532,55.872,53.532,35.116,0.0,2.339,48,89.813,0,0,0,100,0,0,1,0,0,0,0,1,0,1,0,422,477,660.0,1,566,324
1,1,30.814,0.0,21.600,41.276,41.276,30.814,1.0,19.676,61,49.442,0,0,0,90,0,1,1,895,0,0,1,3,0,1,0,162,218,321.0,0,392,564
2,2,18.979,1.0,15.245,18.979,18.979,34.401,1.0,3.735,45,88.452,0,1,1,90,0,1,1,707,0,1,1,3,0,1,1,326,274,122.0,1,2063,1893
3,3,34.941,1.0,34.941,36.863,34.941,35.310,0.0,1.921,47,85.277,0,1,0,100,0,1,1,1399,0,1,1,3,0,1,0,287,394,321.0,0,1590,966
4,4,35.939,0.0,60.025,59.243,60.025,35.939,0.0,-0.782,43,66.679,0,1,0,100,0,1,1,1352,0,1,1,3,0,0,0,504,353,660.0,1,870,782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,2134,31.259,0.0,25.088,35.403,35.403,31.259,1.0,10.315,21,53.298,1,0,0,100,0,1,1,842,0,1,1,3,0,1,1,152,109,321.0,0,561,720
2135,2135,26.999,1.0,23.156,26.999,26.999,30.876,1.0,3.843,17,102.967,1,0,0,100,0,1,1,417,1,1,1,3,0,0,1,373,218,321.0,0,1759,1030
2136,2136,26.937,0.0,46.826,63.878,46.826,26.937,0.0,17.052,53,69.854,1,1,0,90,0,1,1,753,1,1,1,3,0,1,0,419,364,526.0,1,1391,1041
2137,2137,8.773,1.0,21.178,8.773,8.773,27.495,1.0,-12.406,14,60.000,1,0,0,100,0,0,1,0,0,1,0,1,0,0,0,166,169,28.0,1,999,1838


In [15]:
# generate idx split
df_split = pd.DataFrame({'idx': range(len(actg_syn))})
for i in range(10):
    seed = 2025511 + i
    np.random.seed(seed)
    df_split[f'random_idx{i}'] = np.random.permutation(df_split['idx'] )
df_split.to_csv('idx_split_actg_syn.csv', index=False)
df_split.head()

,idx,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
0,0,1269,1396,239,2069,3,1366,316,889,1523,1133
1,1,1996,61,179,1994,2103,149,339,167,1632,1037
2,2,1807,928,677,1964,797,613,50,181,1868,739
3,3,1937,2138,186,1694,1497,111,364,559,1125,425
4,4,64,895,196,248,2134,1749,1868,1572,636,932


In [16]:
print(f'Censoring rate: {(1 - actg_syn.event.mean()) * 100:.2f}%')
print(f'Treatment rate: {actg_syn.W.mean() * 100:.2f}%')
print(f'True ATE: {actg_syn.true_cate.mean()}')
print(f'N: {actg_syn.shape[0]}')
print(f't_max: {actg_syn.observed_time.max()} (month)')

Censoring rate: 51.19%
Treatment rate: 56.15%
True ATE: 5.044362088358642
N: 2139
t_max: 43.44397590668216 (month)
